### Baseline: logistic regression, linear regression, regularized logistic regression, regularized linear regression, sigmoid function
### Dataset Analysis:
the dataset includes cat, dog and other species. additionally, an image may contains two species. 
* we should first do face detection from line segments, to segments grouping in order to form a small region or part of the face, to forming an entire face by grouping based on the regions. in the following layer, the model should be able to identify if the face is one of cat, dog, or wild animals. in this part, there will be at least 3 layers given there will be at least one for line segments, at least one for segment grouping into region, amd at least one for face forming based on regions.
* in this case, in certain step we shoudl identify if the picture contains cat, dog, and e.g. rabbit or no. so the here it should be binary classification for each unit(neuron). e.g. only two classes, cat and dog. output y = [ bool_has_cat, bool_has_dog ]. Hence this layer should be using sigmoid as activation function.
* Then, in the following layer, emotion should be identified to learn that y = [cat happy?, dog happy?, cat sad?, dog sad?, cat angry?, dog angry?, cat relax?, dog relax?] which given each emotional class we should be answering yes or no. if the image is without cat, then all emotional classes relating to cat should be zero. if the image contains cat and dog, emotional classes relating cat and dog should at least have two ones such that cat has certain emotoin relating to a emotion category, and dog should also have certain emotion relating to certain category. it is possible that the specie is happy and sad at the same time such that there are more than two ones. It is due to this that for each layer, ti should again use sigmoid activatoin function.
* given that the chance of having that emotion category is approximated, now this follwoing layer should further learn the level of emotion for each emotional class. for instance, if the image contains cat and dog and that for cat the happy and relaxed are both detected, the two neurons for cat happy and cat relaxed should one compute the level of happy and the other one compute the level of relaxed. the same logic applies to dog. in this case, the function should be ReLU. e.g. y = [level of cat is happy, level of dog is happy, level of cat is sad, level of dog is sad, level of cat is angry, level of dog is angry, level of cat is relaxed, level of dog is relaxed]. if the image contains only cat, then all emtional classes level relating to dog should be zero.
* now this layer should identify based on the level of each emotional class. for instance, let's say, the input to this layer is = [level of cat is happy, level of dog is happy, level of cat is sad, level of dog is sad, level of cat is angry, level of dog is angry, level of cat is relaxed, level of dog is relaxed] and that the image contains both cat and dog. cat is detected as happy and relaxed, dog is detected as happy and sad, the input to this layer is then = [0.5, 0.4, 0.0, 0.5, 0.0, 0.0, 0.7, 0.0], then the output of this layer should contains only two ones being [0, 0, 0, 1, 0, 0, 1, 0] such that the cat is relaxed and dog is sad. this layer hence should use either softmax, or, for each unit, a sigmoid.

In [ ]:
import os
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### 1. Data pre-processing
* Since in the baseline model insteresed implementation should use conventional model, we'll stick with logistic regression, although it's important to note that this approach will possibily yield a very ppor result for such a complex image processing task.
* First, you need to preprocess your images. This involves loading the images, resizing them to a uniform size, converting them to grayscale, and flattening them into vectors.

#### Benefits of Using Grayscale Images
Reduced Complexity: Grayscale images are less complex than color images, making them easier to process with simpler algorithms.
Reduced Computational Load: Grayscale images require less computational power and memory, as they have only one channel compared to three in color images.
Focus on Texture and Shape: Converting to grayscale can help the model focus on the texture and shape information, which might be more relevant for certain tasks like emotion detection in animals.

In [ ]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    file_paths = []  # List to store file paths
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)  # Get the full file path
        img = cv2.imread(file_path)
        if img is not None:
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            img_resized = cv2.resize(gray_img, (32, 32))  # Resize images
            images.append(img_resized.flatten())
            labels.append(label)
            file_paths.append(file_path)  # Store the file path for prediction visualization
    return images, labels, file_paths


### 2. Labeling the Data
If one has dataset for training and is without labels, one will need to assign labels to the data. Since we have separate folders for each emotion and labels for each data, we can skipp this part.
#### Define the path to the sub dataset folders

In [ ]:
dataset_folder = "/kaggle/input/pets-facial-expression-dataset"
# Define the path to the dataset folders
happy_folder = "/kaggle/input/pets-facial-expression-dataset/happy"
sad_folder = "/kaggle/input/pets-facial-expression-dataset/Sad"
angry_folder = "/kaggle/input/pets-facial-expression-dataset/Angry"

#### Load data and Combine data

In [ ]:
# Load data
happy_images, happy_labels = load_images_from_folder(happy_folder, 0)  # Label 0 for happy
sad_images, sad_labels = load_images_from_folder(sad_folder, 1)  # Label 1 for sad
angry_images, angry_labels = load_images_from_folder(angry_folder, 2)  # Label 2 for angry

# Combine data
X = np.array(happy_images + sad_images + angry_images)
y = np.array(happy_labels + sad_labels + angry_labels)

### 3. Splitting the Data
Split dataset into training and testing sets. This is essential for evaluating the performance of the model.
A common split is 80% for training and 20% for testing.

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4. Building the Model
Use logistic regression for the classification. 
As we have more than two emotions, we need to utilize logistic regression in the scikit-learn supports for a multi-class classification setup.

In [ ]:
# Create and train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

### 5. Predict and evaluate

In [ ]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

#### visualizing the prediction result

In [ ]:
import matplotlib.pyplot as plt
import cv2

def visualize_predictions(file_paths, predictions, true_labels=None, num_images=10):
    plt.figure(figsize=(15, 5))
    for i in range(num_images):
        img = cv2.imread(file_paths[i], cv2.IMREAD_GRAYSCALE)
        img_resized = cv2.resize(img, (32, 32))
        plt.subplot(2, num_images // 2, i + 1)
        plt.imshow(img_resized, cmap='gray')
        title = f"Pred: {predictions[i]}"
        if true_labels is not None:
            title += f"\nTrue: {true_labels[i]}"
        plt.title(title)
        plt.axis('off')
    plt.show()


## Limitations
Feature Extraction: This approach uses very basic feature extraction (flattening the image), which might not capture the necessary details for accurate emotion classification.
Model Complexity: Logistic regression is quite basic for image classification tasks.
Data Quality: The quality and size of your dataset will significantly impact the performance of your model.